In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('data/train.csv')
store_df = pd.read_csv('data/store.csv')
test_df = pd.read_csv('data/test.csv')

/Users/neway/software/anaconda3/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 100)
pd.set_option('expand_frame_repr', False)
pd.set_option('precision', 6)

In [3]:
all_df = pd.concat([train_df, test_df])

### Data Clean

In [4]:
def data_clean(df):
    df['Assortment'] = df['Assortment'].astype('str')
    df['StoreType'] = df['StoreType'].astype('str')
    df['StateHoliday'] = df['StateHoliday'].astype('str')

    df['CompetitionDistance'] = df.CompetitionDistance.fillna(0)

    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)
    
    return df

all_df = data_clean(all_df)

### Event Feature Extraction

In [6]:
from datetime import datetime, timedelta
from isoweek import Week

def event_feature_extraction_01(df):
    df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].astype(np.int32)

    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    df["CompetitionOpenDate"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear,month=df.CompetitionOpenSinceMonth, day=15))
    df["DaysAfterCompetitionOpen"] = df.Date.subtract(df.CompetitionOpenDate).dt.days
    
    df.loc[df.DaysAfterCompetitionOpen<0, "DaysAfterCompetitionOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear <= 1990, "DaysAfterCompetitionOpen"] = 0
    
    df["MonthAfterCompetitionOpen"] = df["DaysAfterCompetitionOpen"]//30
    df.loc[df.MonthAfterCompetitionOpen>24, "MonthAfterCompetitionOpen"] = 24
    
    df['Promo2SinceYear'] = df['Promo2SinceYear'].astype(np.int32)
    df['Promo2SinceWeek'] = df['Promo2SinceWeek'].astype(np.int32)
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))

    df["DaysAfterPromo2"] = df.Date.subtract(df["Promo2Since"]).dt.days
    
    df.loc[df.DaysAfterPromo2<0, "DaysAfterPromo2"] = 0
    df.loc[df.Promo2SinceYear <= 1990, "DaysAfterPromo2"] = 0

    df["WeekAfterPromo2"] = df["DaysAfterPromo2"]//7
    df.loc[df.WeekAfterPromo2<0, "WeekAfterPromo2"] = 0
    df.loc[df.WeekAfterPromo2>25, "WeekAfterPromo2"] = 25
    
    return df

event_feature_extraction_01(all_df)

all_df.head()

,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Customers,Date,DayOfWeek,Id,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,SchoolHoliday,StateHoliday,Store,StoreType,data_type,Year,Month,CompetitionOpenDate,DaysAfterCompetitionOpen,MonthAfterCompetitionOpen,Promo2Since,DaysAfterPromo2,WeekAfterPromo2
0,a,1270.0,9,2008,555.0,2015-07-31,5,NaN,1.0,1,0,1,1900,NaN,5263.0,1,0,1,c,1,2015,7,2008-09-15,2510,24,1900-01-01,0,0
1,a,570.0,11,2007,625.0,2015-07-31,5,NaN,1.0,1,1,13,2010,"Jan,Apr,Jul,Oct",6064.0,1,0,2,a,1,2015,7,2007-11-15,2815,24,2010-03-29,1950,25
2,a,14130.0,12,2006,821.0,2015-07-31,5,NaN,1.0,1,1,14,2011,"Jan,Apr,Jul,Oct",8314.0,1,0,3,a,1,2015,7,2006-12-15,3150,24,2011-04-04,1579,25
3,c,620.0,9,2009,1498.0,2015-07-31,5,NaN,1.0,1,0,1,1900,NaN,13995.0,1,0,4,c,1,2015,7,2009-09-15,2145,24,1900-01-01,0,0
4,a,29910.0,4,2015,559.0,2015-07-31,5,NaN,1.0,1,0,1,1900,NaN,4822.0,1,0,5,a,1,2015,7,2015-04-15,107,3,1900-01-01,0,0


In [7]:
def days_diff(stores, dates, is_cmp_dates):
    current_store = 0
    cmp_date = np.datetime64()
    result = []
    for store, date, is_cmp_date in zip(stores, dates, is_cmp_dates):
        if current_store != store:
            current_store = store
            cmp_date = np.datetime64()
        if is_cmp_date:
            cmp_date = date
        try:
            days_diff = np.abs((date - cmp_date)/np.timedelta64(1,'D'))
        except:
            days_diff = -1
        result.append(days_diff)
    return result


def event_feature_extraction_02(df):

    df['isStateHoliday'] = df['StateHoliday'] != '0'
    df['isSchoolHoliday'] = df['SchoolHoliday'] != 0
    df['isPromo'] = df['Promo'] != 0

    sorted_df = df.sort_values(by=['Store', 'Date'])
    sorted_df['daysAfterStateHoliday'] = days_diff(sorted_df['Store'], sorted_df['Date'], sorted_df['isStateHoliday'])
    sorted_df['daysAfterSchoolHoliday'] = days_diff(sorted_df['Store'], sorted_df['Date'], sorted_df['isSchoolHoliday'])
    sorted_df['daysAfterPromo'] = days_diff(sorted_df['Store'], sorted_df['Date'], sorted_df['isPromo'])
    
    sorted_df = sorted_df.sort_values(by=['Store', 'Date'], ascending=[True, False])
    sorted_df['daysBeforeStateHoliday'] = days_diff(sorted_df['Store'], sorted_df['Date'], sorted_df['isStateHoliday'])
    sorted_df['daysBeforeSchoolHoliday'] = days_diff(sorted_df['Store'], sorted_df['Date'], sorted_df['isSchoolHoliday'])
    sorted_df['daysBeforePromo'] = days_diff(sorted_df['Store'], sorted_df['Date'], sorted_df['isPromo'])

    return sorted_df

all_df = event_feature_extraction_02(all_df)

In [8]:
def event_feature_extraction_03(df):
    df['StateHolidayDay'] = df['isStateHoliday'].apply(lambda x: 1 if x else 0)
    columns = ['StateHolidayDay', 'SchoolHoliday', 'Promo']
    df = df.set_index("Date")
    ascending_df = df.sort_index()

    last_7ds = ascending_df[['Store']+columns].groupby("Store").rolling(7, min_periods=1).sum()
    last_7ds.drop('Store',1,inplace=True)
    last_7ds.reset_index(inplace=True)
    last_7ds = last_7ds.rename(columns={'StateHolidayDay':'last_7d_state_holiday', 'SchoolHoliday':'last_7d_school_holiday', 'Promo': 'last_7d_promo'})

    descending_df = df.sort_index(ascending=False)
    coming_7ds = descending_df[['Store']+columns].groupby("Store").rolling(7, min_periods=1).sum()
    coming_7ds.drop('Store',1,inplace=True)
    coming_7ds.reset_index(inplace=True)
    coming_7ds = coming_7ds.rename(columns={'StateHolidayDay':'coming_7d_state_holiday', 'SchoolHoliday':'coming_7d_school_holiday', 'Promo': 'coming_7d_promo'})
        
    return last_7ds, coming_7ds

last_7d_all_df, comint_7d_all_df = event_feature_extraction_03(all_df)

all_df.reset_index(inplace=True)

event_feature_df = all_df.merge(last_7d_all_df, how='left', on=['Store', 'Date'])
event_feature_df = event_feature_df.merge(comint_7d_all_df, how='left', on=['Store', 'Date'])

event_feature_df.head()

,index,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Customers,Date,DayOfWeek,Id,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,SchoolHoliday,StateHoliday,Store,StoreType,data_type,Year,Month,CompetitionOpenDate,DaysAfterCompetitionOpen,MonthAfterCompetitionOpen,Promo2Since,DaysAfterPromo2,WeekAfterPromo2,isStateHoliday,isSchoolHoliday,isPromo,daysAfterStateHoliday,daysAfterSchoolHoliday,daysAfterPromo,daysBeforeStateHoliday,daysBeforeSchoolHoliday,daysBeforePromo,StateHolidayDay,last_7d_state_holiday,last_7d_school_holiday,last_7d_promo,coming_7d_state_holiday,coming_7d_school_holiday,coming_7d_promo
0,0,a,1270.0,9,2008,NaN,2015-09-17,4,1.0,1.0,1,0,1,1900,NaN,NaN,0,0,1,c,3,2015,9,2008-09-15,2558,24,1900-01-01,0,0,False,False,True,105.0,13.0,0.0,-1.0,-1.0,0.0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,856,a,1270.0,9,2008,NaN,2015-09-16,3,857.0,1.0,1,0,1,1900,NaN,NaN,0,0,1,c,3,2015,9,2008-09-15,2557,24,1900-01-01,0,0,False,False,True,104.0,12.0,0.0,-1.0,-1.0,0.0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,1712,a,1270.0,9,2008,NaN,2015-09-15,2,1713.0,1.0,1,0,1,1900,NaN,NaN,0,0,1,c,3,2015,9,2008-09-15,2556,24,1900-01-01,0,0,False,False,True,103.0,11.0,0.0,-1.0,-1.0,0.0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2568,a,1270.0,9,2008,NaN,2015-09-14,1,2569.0,1.0,1,0,1,1900,NaN,NaN,0,0,1,c,3,2015,9,2008-09-15,2555,24,1900-01-01,0,0,False,False,True,102.0,10.0,0.0,-1.0,-1.0,0.0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,3424,a,1270.0,9,2008,NaN,2015-09-13,7,3425.0,0.0,0,0,1,1900,NaN,NaN,0,0,1,c,3,2015,9,2008-09-15,2554,24,1900-01-01,0,0,False,False,False,101.0,9.0,9.0,-1.0,-1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,4.0


In [9]:
event_feature_df = event_feature_df.drop(['index', 'Id'], axis=1)

event_feature_df.to_csv('event_feature.csv', index = False)